In [1]:
# Create a fresh local flowserv instance for test purposes.
from flowserv.client import Flowserv

client = Flowserv(basedir='.flowserv', clear=True)

In [2]:
# Install the 'Hello World' benchmark as a multi-user workflow.
# If a workflow is installed as multi-user we explicitly need to
# create users and user submissions before running the workflow.
workflow_id = client.install('helloworld', multi_user=True, ignore_postproc=False)

In [3]:
# Register users 'aclice' and 'bob'
client.register(username='alice', password='pwd_alice')
client.register(username='bob', password='pwd_bob')

'74a796383a124285aefc62a19445d9f2'

In [4]:
# Before being able to start a workflow run we need to create
# a new submission.

client.login(username='alice', password='pwd_alice')
team_alice = client.create_submission(workflow_id, name='Team Alice')

In [5]:
# Load a workflow object representing the submission for 'Team Alice'.
# This workflow can then be used to submit runs (start_run).
submission = client.submission(workflow_id=workflow_id, group_id=team_alice)

from io import StringIO

names = StringIO('\n'.join(['Alice', 'Bob', 'Claire', 'Dave']))
args = dict({
    'greeting': 'Hey there',
    'sleeptime': 0.2,
    'names': names
})
run = submission.start_run(args)
assert run.is_success()

In [6]:
# Repeat run submission for 'bob'

client.login(username='bob', password='pwd_bob')
team_bob = client.create_submission(workflow_id, name='Team Bob')

In [7]:
# Run workflow for 'Team Bob'.
submission = client.submission(workflow_id=workflow_id, group_id=team_bob)

from io import StringIO

names = StringIO('\n'.join(['Elizabeth-Anne', 'Francisco', 'Georgia', 'Howard', 'Isabella', 'Jacob']))
args = dict({
    'greeting': 'Welcome',
    'sleeptime': 0.2,
    'names': names
})
run = submission.start_run(args)
assert run.is_success()

In [8]:
# Get handle for post-processing results and list the
# available files.
postrun = submission.get_postproc_results()
for f in postrun.files():
    print(f.name)

results/ngrams.csv


In [9]:
# Print results in the ngrams.csv file
with postrun.open_file('results/ngrams.csv') as f:
    for line in f:
        print(line.decode('utf-8').strip())

WEL,6
ELC,6
LCO,6
COM,6
OME,6
HEY,4
THE,4
HER,4
ERE,4
ABE,2
ELI,1
LIZ,1
IZA,1
ZAB,1
BET,1
ETH,1
TH-,1
H-A,1
-AN,1
ANN,1
NNE,1
FRA,1
RAN,1
ANC,1
NCI,1


In [10]:
# Remove all generated files
client.erase()